In [1]:
#AIDI Project V2
#The purpose of this notebook is to implement the proposed method in the SOW2 to generate realistic greenhouse temperatures 
#based on various weather conditions recorded in Windsor Ontario.
#The Data will need to be cleaned and some features need to be prepared to be implemented with the proposed method 
#They are: Outside Temp, Wind Dir, Wind Speed, Wind Gust, and Condition.
#Some of these features will be returned to their original states once they have been used to generate the Inside Temp feature
#They are: Wind Dir and Condition
#Once the we have the generated data points for Inside Temp they will be shifted forward one hour so each row has a refrence to
#the Inside Temp 1 Hour prior. This will be useful when implementing a model since it will be able to uncover a relationship
#between future weather features and a supplied current greenhouse temperature to predict a future greenhouse temperature.

import pandas as pd
import numpy as np

def fahr_to_celsius(temp_fahr):
    """Convert Fahrenheit to Celsius
    
    Return Celsius conversion of input"""
    temp_celsius = (temp_fahr - 32) * 5 / 9
    return round(temp_celsius, 2)

weatherData = pd.read_csv("WeatherData.csv")

In [2]:
display(weatherData)

,Date,Time,Outside Temp,Dew Point,Humidity,Wind Dir,Wind Speed,Wind Gust,Pressure,Precip,Condition,Unnamed: 11
0,2019-07-01,0:00,63 F,59 F,88 %,NE,5 mph,0 mph,29.37 in,0.0 in,Fair,NaN
1,2019-07-01,1:00,63 F,59 F,88 %,ENE,3 mph,0 mph,29.36 in,0.0 in,Fair,NaN
2,2019-07-01,2:00,63 F,59 F,88 %,S,2 mph,0 mph,29.34 in,0.0 in,Fair,NaN
3,2019-07-01,3:00,63 F,59 F,88 %,SSW,3 mph,0 mph,29.34 in,0.0 in,Fair,NaN
4,2019-07-01,4:00,61 F,57 F,88 %,CALM,0 mph,0 mph,29.34 in,0.0 in,Fair,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
2113,2019-08-31,19:00,64 F,57 F,77 %,E,5 mph,0 mph,29.60 in,0.0 in,Cloudy,NaN
2114,2019-08-31,20:00,63 F,57 F,82 %,E,5 mph,0 mph,29.60 in,0.0 in,Cloudy,NaN
2115,2019-08-31,21:00,63 F,54 F,72 %,ENE,10 mph,0 mph,29.60 in,0.0 in,Fair,NaN
2116,2019-08-31,22:00,63 F,54 F,72 %,ENE,8 mph,0 mph,29.60 in,0.0 in,Mostly Cloudy,NaN


In [3]:
#First lets drop the features we will not be using, Wind Gust may be a value we can return to look at if we later want to add 
#more features to the model however the others are not typically forcasted on weather sites so we will not be using them.
#During research they also did not come up as points of influence on internal greenhouse temperature

weatherData = weatherData.drop(['Dew Point', 'Wind Gust', 'Pressure', 'Precip', 'Unnamed: 11'], axis=1)
display(weatherData)

,Date,Time,Outside Temp,Humidity,Wind Dir,Wind Speed,Condition
0,2019-07-01,0:00,63 F,88 %,NE,5 mph,Fair
1,2019-07-01,1:00,63 F,88 %,ENE,3 mph,Fair
2,2019-07-01,2:00,63 F,88 %,S,2 mph,Fair
3,2019-07-01,3:00,63 F,88 %,SSW,3 mph,Fair
4,2019-07-01,4:00,61 F,88 %,CALM,0 mph,Fair
...,...,...,...,...,...,...,...
2113,2019-08-31,19:00,64 F,77 %,E,5 mph,Cloudy
2114,2019-08-31,20:00,63 F,82 %,E,5 mph,Cloudy
2115,2019-08-31,21:00,63 F,72 %,ENE,10 mph,Fair
2116,2019-08-31,22:00,63 F,72 %,ENE,8 mph,Mostly Cloudy


In [4]:
#Now we can start preparing the data to be used in the method which will generate greenhouse temperatures, most of the features 
#need to be converted from strings to floats/ints

weatherData['Outside Temp'] = weatherData['Outside Temp'].map(lambda x: x.replace("F", ""))
weatherData['Outside Temp'] = weatherData['Outside Temp'].map(lambda x: x.strip())
weatherData['Humidity'] = weatherData['Humidity'].map(lambda x: x.replace("%", ""))
weatherData['Humidity'] = weatherData['Humidity'].map(lambda x: x.strip())
weatherData['Humidity'] = weatherData['Humidity'].astype(int)
weatherData['Wind Speed'] = weatherData['Wind Speed'].map(lambda x: x.replace("mph", ""))
weatherData['Wind Speed'] = weatherData['Wind Speed'].map(lambda x: x.strip())
weatherData['Wind Speed'] = weatherData['Wind Speed'].astype(int)
display(weatherData)

,Date,Time,Outside Temp,Humidity,Wind Dir,Wind Speed,Condition
0,2019-07-01,0:00,63,88,NE,5,Fair
1,2019-07-01,1:00,63,88,ENE,3,Fair
2,2019-07-01,2:00,63,88,S,2,Fair
3,2019-07-01,3:00,63,88,SSW,3,Fair
4,2019-07-01,4:00,61,88,CALM,0,Fair
...,...,...,...,...,...,...,...
2113,2019-08-31,19:00,64,77,E,5,Cloudy
2114,2019-08-31,20:00,63,82,E,5,Cloudy
2115,2019-08-31,21:00,63,72,ENE,10,Fair
2116,2019-08-31,22:00,63,72,ENE,8,Mostly Cloudy


In [5]:
#Converting the temperatures to Celsius

weatherData['Outside Temp'] = fahr_to_celsius(pd.to_numeric(weatherData['Outside Temp']))
display(weatherData)

,Date,Time,Outside Temp,Humidity,Wind Dir,Wind Speed,Condition
0,2019-07-01,0:00,17.22,88,NE,5,Fair
1,2019-07-01,1:00,17.22,88,ENE,3,Fair
2,2019-07-01,2:00,17.22,88,S,2,Fair
3,2019-07-01,3:00,17.22,88,SSW,3,Fair
4,2019-07-01,4:00,16.11,88,CALM,0,Fair
...,...,...,...,...,...,...,...
2113,2019-08-31,19:00,17.78,77,E,5,Cloudy
2114,2019-08-31,20:00,17.22,82,E,5,Cloudy
2115,2019-08-31,21:00,17.22,72,ENE,10,Fair
2116,2019-08-31,22:00,17.22,72,ENE,8,Mostly Cloudy


In [6]:
#There are days where many additional recordings were made but since we are doing hourly predictions lets remove them for now, 
#again we can come back and return these values later if the model requires more data.

i = 0
for x in weatherData['Time']:
    if ':00' in x:
        i = i+1
    else:
        weatherData = weatherData.drop(i, axis=0)
        i = i+1

In [7]:
weatherData['Time'].unique()

array(['0:00', '1:00', '2:00', '3:00', '4:00', '5:00', '6:00', '7:00',
       '8:00', '9:00', '10:00', '11:00', '12:00', '13:00', '14:00',
       '15:00', '16:00', '17:00', '18:00', '19:00', '20:00', '21:00',
       '22:00', '23:00'], dtype=object)

In [8]:
display(weatherData)

,Date,Time,Outside Temp,Humidity,Wind Dir,Wind Speed,Condition
0,2019-07-01,0:00,17.22,88,NE,5,Fair
1,2019-07-01,1:00,17.22,88,ENE,3,Fair
2,2019-07-01,2:00,17.22,88,S,2,Fair
3,2019-07-01,3:00,17.22,88,SSW,3,Fair
4,2019-07-01,4:00,16.11,88,CALM,0,Fair
...,...,...,...,...,...,...,...
2113,2019-08-31,19:00,17.78,77,E,5,Cloudy
2114,2019-08-31,20:00,17.22,82,E,5,Cloudy
2115,2019-08-31,21:00,17.22,72,ENE,10,Fair
2116,2019-08-31,22:00,17.22,72,ENE,8,Mostly Cloudy


In [9]:
weatherData["Wind Dir"].unique()

array(['NE', 'ENE', 'S', 'SSW', 'CALM', 'VAR', 'SSE', 'SW', 'WSW', 'W',
       'WNW', 'NW', 'NNW', 'NNE', 'SE', 'N', 'E', 'ESE', nan],
      dtype=object)

In [10]:
#1 row with missing value for wind direction, wind speed was 0 at the time so lets put CALM into this missing value

weatherData["Wind Dir"].isna().sum()

1

In [11]:
weatherData.loc[weatherData['Wind Dir'].isna()]

,Date,Time,Outside Temp,Humidity,Wind Dir,Wind Speed,Condition
413,2019-07-15,11:00,28.89,40,NaN,0,Mostly Cloudy


In [12]:
weatherData.loc[weatherData['Wind Dir'].isna(), 'Wind Dir'] = 'CALM'

In [13]:
weatherData["Wind Dir"].isna().sum()

0

In [14]:
weatherData['Wind Speed'].unique()

array([ 5,  3,  2,  0,  6,  7, 10,  9,  8, 13, 12, 14, 15, 16, 17, 18, 20,
       21, 22, 23, 26])

In [15]:
weatherData['Condition'].unique()

array(['Fair', 'Mostly Cloudy', 'Partly Cloudy', 'Cloudy', 'Light Rain',
       'Thunder in the Vicinity', 'Light Rain with Thunder', 'Mist',
       'Mostly Cloudy / Windy', 'Fair / Windy', 'T-Storm', 'Thunder',
       'Partly Cloudy / Windy', 'Haze', 'Heavy T-Storm', 'Cloudy / Windy'],
      dtype=object)

In [16]:
#No missing values for wind speed or condition
#Changing time to an integer temporarilly

weatherData['Time'] = weatherData['Time'].map(lambda x: x.replace(":00", ""))

#For the predictions we want datapoints during hours of sunlight which are from 6AM to 9PM in the July - August months.
#Remove rows where the time is outside this range.

for index, row in weatherData.iterrows():
        weatherData.loc[index, 'Time'] = int(row['Time'])    

weatherData.drop(weatherData[weatherData.Time < 6].index, inplace=True)
weatherData.drop(weatherData[weatherData.Time > 21].index, inplace=True)

display(weatherData)

,Date,Time,Outside Temp,Humidity,Wind Dir,Wind Speed,Condition
6,2019-07-01,6,17.22,88,ENE,3,Mostly Cloudy
7,2019-07-01,7,20.00,78,SSE,6,Fair
8,2019-07-01,8,23.89,69,SSW,7,Fair
9,2019-07-01,9,26.11,54,SW,10,Partly Cloudy
10,2019-07-01,10,26.11,54,SW,9,Partly Cloudy
...,...,...,...,...,...,...,...
2111,2019-08-31,17,20.00,52,E,7,Cloudy
2112,2019-08-31,18,18.89,68,E,6,Cloudy
2113,2019-08-31,19,17.78,77,E,5,Cloudy
2114,2019-08-31,20,17.22,82,E,5,Cloudy


In [17]:
#Now lets make a copy before we convert the Wind Dir feature and Condition feature to numeric values.
#For more information on how this conversion works check the SOW2.

weatherData0 = weatherData.copy()

for index, row in weatherData.iterrows():
    if row['Condition'] == 'Fair' or row['Condition'] == 'Fair / Windy':
        weatherData.loc[index, 'Condition'] = 1
    elif row['Condition'] == 'Haze' or row['Condition'] == 'Mist':
        weatherData.loc[index, 'Condition'] = 1.5
    elif row['Condition'] == 'Partly Cloudy' or row['Condition'] == 'Light Rain' or row['Condition'] == 'Partly Cloudy / Windy':
        weatherData.loc[index, 'Condition'] = 2
    elif row['Condition'] == 'Thunder in the Vicinity' or row['Condition'] == 'Light Rain with Thunder':
        weatherData.loc[index, 'Condition'] = 2.5
    elif row['Condition'] == 'Mostly Cloudy' or row['Condition'] == 'Mostly Cloudy / Windy':
        weatherData.loc[index, 'Condition'] = 3
    elif row['Condition'] == 'Thunder':
        weatherData.loc[index, 'Condition'] = 3.5
    elif row['Condition'] == 'Cloudy' or row['Condition'] == 'T-Storm' or row['Condition'] == 'Cloudy / Windy':
        weatherData.loc[index, 'Condition'] = 4
    else:
        weatherData.loc[index, 'Condition'] = 4.5
        
for index, row in weatherData.iterrows():
    if row['Wind Dir'] == 'N' or row['Wind Dir'] == 'S':
        weatherData.loc[index, 'Wind Dir'] = 1
    elif row['Wind Dir'] == 'E' or row['Wind Dir'] == 'W':
        weatherData.loc[index, 'Wind Dir'] = 2
    elif row['Wind Dir'] == 'ENE' or row['Wind Dir'] == 'VAR' or row['Wind Dir'] == 'WSW' or row['Wind Dir'] == 'WNW' or row['Wind Dir'] == 'ESE':
        weatherData.loc[index, 'Wind Dir'] = 2.5
    elif row['Wind Dir'] == 'SSW' or row['Wind Dir'] == 'SSE' or row['Wind Dir'] == 'NNW' or row['Wind Dir'] == 'NNE':
        weatherData.loc[index, 'Wind Dir'] = 3
    elif row['Wind Dir'] == 'NE' or row['Wind Dir'] == 'SW' or row['Wind Dir'] == 'NW' or row['Wind Dir'] == 'SE':
        weatherData.loc[index, 'Wind Dir'] = 3.5
    else:
        weatherData.loc[index, 'Wind Dir'] = 0



display(weatherData)

,Date,Time,Outside Temp,Humidity,Wind Dir,Wind Speed,Condition
6,2019-07-01,6,17.22,88,2.5,3,3
7,2019-07-01,7,20.00,78,3,6,1
8,2019-07-01,8,23.89,69,3,7,1
9,2019-07-01,9,26.11,54,3.5,10,2
10,2019-07-01,10,26.11,54,3.5,9,2
...,...,...,...,...,...,...,...
2111,2019-08-31,17,20.00,52,2,7,4
2112,2019-08-31,18,18.89,68,2,6,4
2113,2019-08-31,19,17.78,77,2,5,4
2114,2019-08-31,20,17.22,82,2,5,4


In [18]:
#Now the features are prepared to be used in the generation of the Inside Temp feature.

import random
insideTemp = []

for index, row in weatherData.iterrows():
        insideTemp.append(row['Outside Temp'] + (random.randrange(9,11) / ((((12 - int(row['Time'])) ** 2) / 4) + 1)) * (np.sqrt(1/row['Condition'])) + row['Outside Temp'] * (1 - np.sqrt(row['Wind Speed'])) * row['Wind Dir'] / 10 / random.randrange(30, 51))
        


In [19]:
display(insideTemp)

[17.649572992939692,
 21.115766755230645,
 25.63904043305532,
 27.80376257782196,
 29.151560932959764,
 34.457123619590966,
 37.11511671406311,
 37.61702127659574,
 34.50717967697245,
 33.71758715092143,
 32.215754966142136,
 32.81560344807213,
 31.31230307981208,
 30.525399129172715,
 27.693779126967605,
 27.28449620637256,
 24.06801625279304,
 26.62540122588727,
 28.268615440234885,
 30.59523890146237,
 31.8024740886358,
 32.51363248716391,
 32.766166405764196,
 33.85218408404094,
 25.541034935779525,
 22.302845105058474,
 22.16490740668219,
 22.75578942967249,
 23.964048809066902,
 24.58231344160506,
 26.829567534416597,
 28.947156688911402,
 30.861333333333334,
 31.05582654245181,
 29.390607715535452,
 29.660438179242785,
 29.224400000000003,
 28.820685641324594,
 28.404574541253265,
 27.37103966075158,
 24.277638361355745,
 22.872733398240722,
 21.455738201345802,
 23.825840223348685,
 24.577989939050468,
 27.36430357675315,
 28.495428067409335,
 33.10068534198228,
 35.43900505894

In [20]:
#Add this feature to the copy of the datafram we made earlier which preserved the non numeric Wind Dir and Condition features

weatherData0['Inside Temp'] = insideTemp
weatherData0['Inside Temp'] = weatherData0['Inside Temp'].round(2)

#Also fix the format of date and time so it can be recognized by tableau as a date/time feature

for index, row in weatherData0.iterrows():
        weatherData0.loc[index, 'Time'] = str(row['Time']) + ':00:00'

display(weatherData0)

,Date,Time,Outside Temp,Humidity,Wind Dir,Wind Speed,Condition,Inside Temp
6,2019-07-01,6:00:00,17.22,88,ENE,3,Mostly Cloudy,17.65
7,2019-07-01,7:00:00,20.00,78,SSE,6,Fair,21.12
8,2019-07-01,8:00:00,23.89,69,SSW,7,Fair,25.64
9,2019-07-01,9:00:00,26.11,54,SW,10,Partly Cloudy,27.80
10,2019-07-01,10:00:00,26.11,54,SW,9,Partly Cloudy,29.15
...,...,...,...,...,...,...,...,...
2111,2019-08-31,17:00:00,20.00,52,E,7,Cloudy,20.48
2112,2019-08-31,18:00:00,18.89,68,E,6,Cloudy,19.22
2113,2019-08-31,19:00:00,17.78,77,E,5,Cloudy,17.99
2114,2019-08-31,20:00:00,17.22,82,E,5,Cloudy,17.35


In [21]:
weatherData0['Date'] = pd.to_datetime(weatherData0.Date, format='%Y-%m-%d')
weatherData0['Date'] = weatherData0['Date'].dt.strftime('%m/%d/%Y')
display(weatherData0)

,Date,Time,Outside Temp,Humidity,Wind Dir,Wind Speed,Condition,Inside Temp
6,07/01/2019,6:00:00,17.22,88,ENE,3,Mostly Cloudy,17.65
7,07/01/2019,7:00:00,20.00,78,SSE,6,Fair,21.12
8,07/01/2019,8:00:00,23.89,69,SSW,7,Fair,25.64
9,07/01/2019,9:00:00,26.11,54,SW,10,Partly Cloudy,27.80
10,07/01/2019,10:00:00,26.11,54,SW,9,Partly Cloudy,29.15
...,...,...,...,...,...,...,...,...
2111,08/31/2019,17:00:00,20.00,52,E,7,Cloudy,20.48
2112,08/31/2019,18:00:00,18.89,68,E,6,Cloudy,19.22
2113,08/31/2019,19:00:00,17.78,77,E,5,Cloudy,17.99
2114,08/31/2019,20:00:00,17.22,82,E,5,Cloudy,17.35


In [22]:
#Shift the Inside temp values so each row has a reference of the inside temp 1 hour earlier (This is we removed the rows 
#earlier that weren't on the hour)

prevTemps = weatherData0['Inside Temp']
prevTemps = prevTemps[:-1]
addTemps = [16]
addTemps[1:] = prevTemps
weatherData0['Inside Temp 1Hr Ago'] = addTemps
display(weatherData0)


,Date,Time,Outside Temp,Humidity,Wind Dir,Wind Speed,Condition,Inside Temp,Inside Temp 1Hr Ago
6,07/01/2019,6:00:00,17.22,88,ENE,3,Mostly Cloudy,17.65,16.00
7,07/01/2019,7:00:00,20.00,78,SSE,6,Fair,21.12,17.65
8,07/01/2019,8:00:00,23.89,69,SSW,7,Fair,25.64,21.12
9,07/01/2019,9:00:00,26.11,54,SW,10,Partly Cloudy,27.80,25.64
10,07/01/2019,10:00:00,26.11,54,SW,9,Partly Cloudy,29.15,27.80
...,...,...,...,...,...,...,...,...,...
2111,08/31/2019,17:00:00,20.00,52,E,7,Cloudy,20.48,21.98
2112,08/31/2019,18:00:00,18.89,68,E,6,Cloudy,19.22,20.48
2113,08/31/2019,19:00:00,17.78,77,E,5,Cloudy,17.99,19.22
2114,08/31/2019,20:00:00,17.22,82,E,5,Cloudy,17.35,17.99


In [24]:
#Data is preped, output the dataframe to be used with tableau to gather insights.

weatherData0.to_csv(r'C:\Users\Gavo_\Documents\AIDI_1002\WeatherOut.csv', index = False)